In [ ]:
"""
Ce code vise à estimer les valeurs manquantes du TCH (Taux de Charge des filières régionales) pour les années 2013 à 2019,
en s'appuyant sur les données disponibles fournies par RTE pour la période 2020 à 2023, et à analyser les écarts éventuels

Created on Wed Jul  3 13:49:08 2024

@author: Thierry ALLEM
"""


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import statsmodels.api as sm
from scipy.stats import hmean

In [ ]:
import warnings
# Pour ignorer tous les warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importation du fichier eco2mix_regional_def préparé, incluant les tco et TCH communiqués par RTE
df_eco2_tch_rte = pd.read_csv('df_eco2_cap_ind_fusion.csv',sep = ';', encoding='latin-1')


In [ ]:
df_eco2_tch_rte.head()

In [ ]:
df_eco2_tch_rte.info()

In [ ]:
df_eco2_tch_rte.isna().sum()

In [ ]:
# Calcul des capacités maximales de production par filière

In [ ]:
# Liste des filières
filieres = ['thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'bioenergies']

# Création des colonnes cmax_filiere_rte
for filiere in filieres:
    tch_col = f'tch_{filiere}_rte'
    cmax_col = f'cmax_{filiere}_rte'
    prod_col = filiere
    
    df_eco2_tch_rte[cmax_col] = df_eco2_tch_rte.apply(
        lambda row: (row[prod_col] / row[tch_col] * 100) if row[prod_col] > 0 and row[tch_col] != 0 else 0,axis=1)

# Affichage du DataFrame mis à jour
df_eco2_tch_rte.head()

In [ ]:
df_eco2_tch_rte.info()

In [ ]:
# +++++++++++++++++++++++++  ANALYSE DES DONNEES DES ANNEEES 2020 A 2023 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
#  FILTRAGE DES LIGNES OU SEULES LES TCH COMMUNIQUES PAR RTE SONT RENSEIGNES (2020 à 2023) (c'est à dire à partir du 01/01/2020)

In [ ]:
# Définition de la date de début pour la sélection (définie d'après les heatmaps des NaN)
# date_debut = pd.Timestamp('2020-01-01')
date_debut = '2020-01-01'

# Filtrage du DataFrame pour ne garder que les lignes où 'Date' est supérieure ou égale à date_debut
df_eco2_tch_rte_20_23 = df_eco2_tch_rte[df_eco2_tch_rte['date'] >= date_debut]
df_eco2_tch_rte_20_23.info()

In [ ]:
# Enregistrement du nouveau DataFrame dans un fichier Excel
df_eco2_tch_rte_20_23.to_excel('eco2_TCH_presents_filtered.xlsx', index=False)

# Enregistrement du nouveau DataFrame dans un fichier CSV
df_eco2_tch_rte_20_23.to_csv('eco2_TCH_presents_filtered.csv', index=False)

In [ ]:
# BREVES STATISTIQUES SUR DONNEES 2020 à 2023 VENTILEES PAR REGION

In [ ]:
# Variables à traiter
variables = ['cmax_thermique_rte', 'cmax_nucleaire_rte', 'cmax_eolien_rte', 'cmax_solaire_rte', 'cmax_hydraulique_rte', 'cmax_bioenergies_rte']

# Fonction pour calculer les statistiques
def calculate_statistics(group):
    stats = {}
    for var in variables:
        if var in group.columns:
            data = group[var].dropna()
            if len(data) > 0:
                stats[f'{var}_mean'] = data.mean()
                stats[f'{var}_median'] = data.median()
                stats[f'{var}_max'] = data.max()
                stats[f'{var}_min'] = data.min()
                stats[f'{var}_std'] = data.std()
                stats[f'{var}_hmean'] = hmean(data) if all(data > 0) else None  # moyenne harmonique (doit être positive)
                stats[f'{var}_iqr'] = data.quantile(0.75) - data.quantile(0.25)  # étendue interquartile
        else:
            print(f"Colonne {var} non trouvée dans le DataFrame")
    return pd.Series(stats)

# Groupement par région et calcul des statistiques
statistics_df = df_eco2_tch_rte_20_23.groupby('region').apply(calculate_statistics)

# Réorganisation du DataFrame pour avoir les régions en colonnes
statistics_df = statistics_df.transpose()

# Exportation des résultats dans un fichier Excel
statistics_df.to_excel('statistiques_par_region.xlsx', index=True)

In [ ]:
# Affichage des distributions sur BBOX

# Variables à analyser
variables = ['cmax_thermique_rte', 'cmax_nucleaire_rte', 'cmax_eolien_rte', 'cmax_solaire_rte', 'cmax_hydraulique_rte', 'cmax_bioenergies_rte']

# Extraction de la filière de la colonne
filiere_map = {
    'cmax_thermique_rte': 'thermique',
    'cmax_nucleaire_rte': 'nucleaire',
    'cmax_eolien_rte': 'eolien',
    'cmax_solaire_rte': 'solaire',
    'cmax_hydraulique_rte': 'hydraulique',
    'cmax_bioenergies_rte': 'bioenergies'
}

# Préparation du DataFrame pour la visualisation
df_long = pd.melt(df_eco2_tch_rte_20_23, id_vars=['region'], value_vars=variables,
                  var_name='Variable', value_name='Value')
df_long['Filiere'] = df_long['Variable'].map(filiere_map)

# Filtrage des valeurs non nulles
df_long = df_long[df_long['Value'].notnull()]

# Visualisation et sauvegarder des distributions par filière et région
for filiere in df_long['Filiere'].unique():
    plt.figure(figsize=(15, 10))
    sns.boxplot(data=df_long[df_long['Filiere'] == filiere], x='region', y='Value')
    plt.title(f'Distributions des valeurs pour la filière {filiere}')
    plt.xlabel('Région')
    plt.ylabel('Valeur')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(f'distributions_{filiere}.png')
    plt.show()

print("Les graphiques ont été exportés avec succès.")

In [ ]:
# Affichage des distributions sur courbes

# Variables à analyser
variables = ['cmax_thermique_rte', 'cmax_nucleaire_rte', 'cmax_eolien_rte', 'cmax_solaire_rte', 'cmax_hydraulique_rte', 'cmax_bioenergies_rte']

# Extraction de la filière de la colonne
filiere_map = {
    'cmax_thermique_rte': 'thermique',
    'cmax_nucleaire_rte': 'nucleaire',
    'cmax_eolien_rte': 'eolien',
    'cmax_solaire_rte': 'solaire',
    'cmax_hydraulique_rte': 'hydraulique',
    'cmax_bioenergies_rte': 'bioenergies'
}

# Préparation du DataFrame pour la visualisation
df_long = pd.melt(df_eco2_tch_rte_20_23, id_vars=['region'], value_vars=variables,
                  var_name='Variable', value_name='Value')
df_long['Filiere'] = df_long['Variable'].map(filiere_map)

# Filtrage des valeurs non nulles
df_long = df_long[df_long['Value'].notnull()]

# Visualisation et sauvegarde des distributions par filière et région
for filiere in df_long['Filiere'].unique():
    # Boxplot
    plt.figure(figsize=(15, 10))
    sns.boxplot(data=df_long[df_long['Filiere'] == filiere], x='region', y='Value')
    plt.title(f'Distributions des valeurs pour la filière {filiere} - Boxplot')
    plt.xlabel('Région')
    plt.ylabel('Valeur')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(f'distributions_boxplot_{filiere}.png')
    plt.show()
    
    # Density Plot
    plt.figure(figsize=(15, 10))
    for region in df_long['region'].unique():
        sns.kdeplot(df_long[(df_long['Filiere'] == filiere) & (df_long['region'] == region)]['Value'], 
                    label=region, shade=True)
    plt.title(f'Distributions des valeurs pour la filière {filiere} - Courbes de densité')
    plt.xlabel('Valeur')
    plt.ylabel('Densité')
    plt.legend(title='Région')
    plt.tight_layout()
    plt.savefig(f'distributions_density_{filiere}.png')
    plt.show()

print("Les graphiques ont été exportés avec succès.")

In [ ]:
# Création d'un writer Excel pour enregistrer les résultats
with pd.ExcelWriter('df_eco2_tch_rte_20_23_stat.xlsx', engine='openpyxl', mode='w') as writer:
    
    regions = df_eco2_tch_rte_20_23['region'].unique()
    
    for region in regions:
        # Filtrage du DataFrame par région
        df_region = df_eco2_tch_rte_20_23[df_eco2_tch_rte_20_23['region'] == region]

        # Statistiques descriptives
        stats = df_region.describe()

        # Écriture des résultats dans une feuille Excel
        stats.to_excel(writer, sheet_name=f'Region_{region}')


In [ ]:
# Filtrage des colonnes qui contiennent 'cmax'
cmax_columns = [col for col in df_eco2_tch_rte_20_23.columns if 'cmax' in col]

# BOXPLOTS PAR REGION
regions = df_eco2_tch_rte_20_23['region'].unique()
for region in regions:
    df_region = df_eco2_tch_rte_20_23[df_eco2_tch_rte_20_23['region'] == region]
    df_cmax = df_region[cmax_columns]
    
    plt.figure(figsize=(10, 6))
    df_cmax.boxplot()
    plt.title(f'Boxplots pour la région {region}')
    plt.savefig(f'boxplot_region_{region}.png')
    plt.show()

In [ ]:
# BOXPLOTS PAR REGION ET ANNÉE
years = df_eco2_tch_rte_20_23['annee'].unique()
for region in regions:
    for year in years:
        df_region_year = df_eco2_tch_rte_20_23[
            (df_eco2_tch_rte_20_23['region'] == region) & (df_eco2_tch_rte_20_23['annee'] == year)
        ]
        df_cmax = df_region_year[cmax_columns]
        
        if not df_cmax.empty:
            plt.figure(figsize=(10, 6))
            df_cmax.boxplot()
            plt.title(f'Boxplots pour la région {region} - Année {year}')
            plt.savefig(f'boxplot_region_{region}_year_{year}.png')
            plt.show()

In [ ]:
# Filtrage des colonnes qui contiennent 'cmax'
cmax_columns = [col for col in df_eco2_tch_rte_20_23.columns if 'cmax' in col]
regions = df_eco2_tch_rte_20_23['region'].unique()

for region in regions:
    df_region = df_eco2_tch_rte_20_23[df_eco2_tch_rte_20_23['region'] == region]
    df_cmax = df_region[cmax_columns]

    plt.figure(figsize=(15, 5 * len(cmax_columns)))
    for i, column in enumerate(cmax_columns, 1):
        plt.subplot(len(cmax_columns), 1, i)
        sm.qqplot(df_cmax[column].dropna(), line='s', ax=plt.gca())
        plt.title(f'Q-Q plot pour la colonne {column} - Région {region}')

    plt.tight_layout()
    plt.savefig(f'qqplot_region_{region}.png')
    plt.show()


In [ ]:
#++++++++++++++++++++       REMPLACEMENT DES VALEURS DE CMAX DE LA PERIODE  2013 A 2019 AVEC MEDIANE,OU MOYENNE CALCULEES SUR LA 2020-2023, OU 0

In [ ]:
 # Définition des filières et colonnes correspondantes
filiere = {
    'thermique': 'thermique',
    'nucleaire': 'nucleaire',
    'eolien': 'eolien',
    'solaire': 'solaire',
    'hydraulique': 'hydraulique',
    'bioenergies': 'bioenergies'
}

colonnes_filiere = {
    'thermique': 'cmax_thermique_rte',
    'nucleaire': 'cmax_nucleaire_rte',
    'eolien': 'cmax_eolien_rte',
    'solaire': 'cmax_solaire_rte',
    'hydraulique': 'cmax_hydraulique_rte',
    'bioenergies': 'cmax_bioenergies_rte'
}

annees_connues = [2020, 2021, 2022, 2023]
annees_inconnues = [2013, 2014, 2015, 2016, 2017, 2018, 2019]

criteres_remplacement = {
    ('AUVERGNE RHONE ALPES', 'thermique'): 'mediane',
    ('AUVERGNE RHONE ALPES', 'nucleaire'): 'mediane',
    ('AUVERGNE RHONE ALPES', 'eolien'): 'mediane',
    ('AUVERGNE RHONE ALPES', 'solaire'): 'mediane',
    ('AUVERGNE RHONE ALPES', 'hydraulique'): 'mediane',
    ('AUVERGNE RHONE ALPES', 'bioenergies'): 'mediane',
    ('BOURGOGNE FRANCHE COMTE', 'thermique'): 'mediane',
    ('BOURGOGNE FRANCHE COMTE', 'nucleaire'): 0,
    ('BOURGOGNE FRANCHE COMTE', 'eolien'): 'mediane',
    ('BOURGOGNE FRANCHE COMTE', 'solaire'): 'moyenne',
    ('BOURGOGNE FRANCHE COMTE', 'hydraulique'): 'mediane',
    ('BOURGOGNE FRANCHE COMTE', 'bioenergies'): 'mediane',
    ('BRETAGNE', 'thermique'): 'moyenne',
    ('BRETAGNE', 'nucleaire'): 0,
    ('BRETAGNE', 'eolien'): 'mediane',
    ('BRETAGNE', 'solaire'): 'moyenne',
    ('BRETAGNE', 'hydraulique'): 'mediane',
    ('BRETAGNE', 'bioenergies'): 'mediane',
    ('CENTRE VAL DE LOIRE', 'thermique'): 'moyenne',
    ('CENTRE VAL DE LOIRE', 'nucleaire'): 0,
    ('CENTRE VAL DE LOIRE', 'eolien'): 'mediane',
    ('CENTRE VAL DE LOIRE', 'solaire'): 'moyenne',
    ('CENTRE VAL DE LOIRE', 'hydraulique'): 'moyenne',
    ('CENTRE VAL DE LOIRE', 'bioenergies'): 'moyenne',
    ('GRAND EST', 'thermique'): 'mediane',
    ('GRAND EST', 'nucleaire'): 'mediane',
    ('GRAND EST', 'eolien'): 'mediane',
    ('GRAND EST', 'solaire'): 'mediane',
    ('GRAND EST', 'hydraulique'): 'mediane',
    ('GRAND EST', 'bioenergies'): 'mediane',
    ('HAUTS DE FRANCE', 'thermique'): 'mediane',
    ('HAUTS DE FRANCE', 'nucleaire'): 'mediane',
    ('HAUTS DE FRANCE', 'eolien'): 'mediane',
    ('HAUTS DE FRANCE', 'solaire'): 'moyenne',
    ('HAUTS DE FRANCE', 'hydraulique'): 'moyenne',
    ('HAUTS DE FRANCE', 'bioenergies'): 'mediane',
    ('ILE DE FRANCE', 'thermique'): 'moyenne',
    ('ILE DE FRANCE', 'nucleaire'): 0,
    ('ILE DE FRANCE', 'eolien'): 'mediane',
    ('ILE DE FRANCE', 'solaire'): 'moyenne',
    ('ILE DE FRANCE', 'hydraulique'): 'moyenne',
    ('ILE DE FRANCE', 'bioenergies'): 'mediane',
    ('NORMANDIE', 'thermique'): 'moyenne',
    ('NORMANDIE', 'nucleaire'): 'mediane',
    ('NORMANDIE', 'eolien'): 'mediane',
    ('NORMANDIE', 'solaire'): 'moyenne',
    ('NORMANDIE', 'hydraulique'): 'moyenne',
    ('NORMANDIE', 'bioenergies'): 'moyenne',
    ('NOUVELLE AQUITAINE', 'thermique'): 'mediane',
    ('NOUVELLE AQUITAINE', 'nucleaire'): 'mediane',
    ('NOUVELLE AQUITAINE', 'eolien'): 'mediane',
    ('NOUVELLE AQUITAINE', 'solaire'): 'moyenne',
    ('NOUVELLE AQUITAINE', 'hydraulique'): 'mediane',
    ('NOUVELLE AQUITAINE', 'bioenergies'): 'mediane',
    ('OCCITANIE', 'thermique'): 'mediane',
    ('OCCITANIE', 'nucleaire'): 'mediane',
    ('OCCITANIE', 'eolien'): 'mediane',
    ('OCCITANIE', 'solaire'): 'mediane',
    ('OCCITANIE', 'hydraulique'): 'mediane',
    ('OCCITANIE', 'bioenergies'): 'mediane',
    ('PAYS DE LA LOIRE', 'thermique'): 'mediane',
    ('PAYS DE LA LOIRE', 'nucleaire'): 0,
    ('PAYS DE LA LOIRE', 'eolien'): 'mediane',
    ('PAYS DE LA LOIRE', 'solaire'): 'mediane',
    ('PAYS DE LA LOIRE', 'hydraulique'): 'moyenne',
    ('PAYS DE LA LOIRE', 'bioenergies'): 'mediane',
    ('PROVENCE ALPES COTE D AZUR', 'thermique'): 'mediane',
    ('PROVENCE ALPES COTE D AZUR', 'nucleaire'): 0,
    ('PROVENCE ALPES COTE D AZUR', 'eolien'): 'mediane',
    ('PROVENCE ALPES COTE D AZUR', 'solaire'): 'mediane',
    ('PROVENCE ALPES COTE D AZUR', 'hydraulique'): 'mediane',
    ('PROVENCE ALPES COTE D AZUR', 'bioenergies'): 'mediane',
}

# Calcul des statistiques pour les années connues
stats = {}
for (region, filiere), method in criteres_remplacement.items():
    filiere_col = colonnes_filiere[filiere]
    data = df_eco2_tch_rte.loc[(df_eco2_tch_rte['region'] == region) & (df_eco2_tch_rte['annee'].isin(annees_connues)), filiere_col]
    if method == 'mediane':
        stats[(region, filiere, 'mediane')] = data.median()
    elif method == 'moyenne':
        stats[(region, filiere, 'moyenne')] = data.mean()

# Remplacement des valeurs des cmax_filiere_rte pour les années inconnues
for (region, filiere), method in criteres_remplacement.items():
    filiere_col = colonnes_filiere[filiere]
    stat_value = stats.get((region, filiere, method), None)
    if stat_value is not None:
        mask = (df_eco2_tch_rte['region'] == region) & (df_eco2_tch_rte['annee'].isin(annees_inconnues))
        df_eco2_tch_rte.loc[mask & (df_eco2_tch_rte[filiere] == 0), filiere_col] = 0
        df_eco2_tch_rte.loc[mask & (df_eco2_tch_rte[filiere] != 0), filiere_col] = stat_value


# Calcul des TCH d'après les cmax évalués précedemment

# Filtration des lignes pour les années entre 2013 et 2019 incluses
df_filtered = df_eco2_tch_rte[(df_eco2_tch_rte['annee'] >= 2013) & (df_eco2_tch_rte['annee'] <= 2019)]


for col in ['thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'bioenergies']:
    df_filtered['tch_' +col +  '_rte'] = df_filtered.apply(
        lambda row: row[col] / (row['cmax_'+col+'_rte'])* 100 if row['cmax_'+col+'_rte'] > 0 else 0,
        axis=1
 )

In [ ]:
df_eco2_tch_rte.head()

In [ ]:
df_eco2_tch_rte.info()

In [ ]:
# Concatenation des résultats 2013-2019 aux résultats 2020-2023

df_eco2_tch_rte_2020_2023 = df_eco2_tch_rte[(df_eco2_tch_rte['annee'] >= 2020) & (df_eco2_tch_rte['annee'] <= 2023)]

df_eco2_tch_rte = pd.concat([df_filtered, df_eco2_tch_rte_2020_2023])

In [ ]:
# **********************************************************************************************************************************************
# Sauvegarde du dataframe incluant les cmax et TCH estimés sur 2013-2019
df_eco2_tch_rte.to_csv('df_eco2_tch_rte_extrapole.csv', index=False)

df_eco2_tch_rte.isna().sum().sum()

In [ ]:
# COMPARAISON DES VALEURS DES CAPACITES DE PRODUCTION EXISTANTES DANS ECO2MIX AVEC CELLES CALCULEES


In [ ]:
#  -------------------            Affichage de toutes les filières par région sur les mêmes visualisations -------------


# Conversion de la colonne 'date'  au format datetime
df_eco2_tch_rte['date'] = pd.to_datetime(df_eco2_tch_rte['date'])

# Création de la colonne 'jour_numero'
df_eco2_tch_rte['jour_numero'] = df_eco2_tch_rte['date'].dt.dayofyear

# Liste des filières et des colonnes correspondantes à tracer
filières = {
    'Thermique': ['cmax_thermique_rte', 'cap_prod_max_thermique', 'ind_prod_thermique', 'thermique_p_disp'],
    'Nucléaire': ['cmax_nucleaire_rte', 'cap_prod_max_nucleaire', 'ind_prod_nucleaire', 'nucleaire_p_disp'],
    'Éolien': ['cmax_eolien_rte', 'eolien_p_disp'],
    'Solaire': ['cmax_solaire_rte', 'solaire_p_disp'],
    'Hydraulique': ['cmax_hydraulique_rte', 'cap_prod_max_hydraulique', 'ind_prod_hydraulique', 'hydraulique_p_disp'],
    'Bioénergies': ['cmax_bioenergies_rte', 'cap_prod_max_bioenergies', 'ind_prod_bioenergies', 'bioenergies_p_disp']
}

# Fonction pour ajouter les barres de saisons
def barre_saisons(ax):
    saisons = {'Hiver': (1, 79),
               'Printemps': (80, 171),
               'Été': (172, 265),
               'Automne': (266, 354),
               'Hiver (suite)': (355, 365)}
    colors = ['lightblue', 'lightgreen', 'lightyellow', 'orange', 'lightblue']
    for (saison, (start, end)), color in zip(saisons.items(), colors):
        ax.axvspan(start, end, color=color, alpha=0.3)
        ax.text((start + end) / 2, ax.get_ylim()[1] * 0.9, saison, ha='center', va='top', fontsize=10, color='gray')

# Liste des régions
regions = df_eco2_tch_rte['region'].unique()

# Calcul du nombre total de jours de 2013 à 2022
jours_total = (pd.Timestamp('2022-12-31') - pd.Timestamp('2013-01-01')).days + 1

# Pour chaque région
for region in regions:
    # Création d'une figure avec un sous-graphiques par filière
    fig, axs = plt.subplots(len(filières), 1, figsize=(20, 5 * len(filières)), sharex=True)
    fig.subplots_adjust(hspace=0.4)  # Espace entre les sous-graphiques
    
    # Pour chaque filière
    for idx, (filiere, colonnes) in enumerate(filières.items()):
        ax = axs[idx]
        # Pour chaque colonne dans la filière
        for col in colonnes:
            # Sélection des données pour la région et la filière actuelle
            data = df_eco2_tch_rte[df_eco2_tch_rte['region'] == region]
            
            if not data.empty:
                # Calcul de la moyenne par jour pour toutes les années combinées
                data['jour_total'] = (data['date'] - pd.Timestamp('2013-01-01')).dt.days + 1
                moyennes = data.groupby('jour_total')[col].mean()
                
                # Ajout de la courbe au graphique
                ax.plot(moyennes.index, moyennes, label=f'{col}')
        
        # Ajout des titres et étiquettes
        ax.set_title(f'Capacité maximale journalière moyenne de production ({filiere}) pour la région {region}')
        ax.set_ylabel("Capacité maximale journalière moyenne (MW)")
        ax.set_xlim(0, jours_total)  # Fixation des limites des abscisses
        ax.grid(True, which='both', linestyle='--', linewidth=0.5, color='gray')  # Ajout une grille gris pâle
        barre_saisons(ax)
        
        # Configuration de l'axe des abscisses pour chaque sous-graphiques pour éviter la surcharge
        ax.set_xticks(range(0, jours_total, 365))
        ax.set_xticklabels([f'Année {2013 + i}' for i in range(0, jours_total // 365 + 1)], rotation=45)
        ax.set_xlabel("Jour total depuis 2013")
    
    # Création d'une légende unique par sous-graphiques
    for ax in axs:
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles, labels, loc='upper right', ncol=1, title="Colonne")

    # Sauvegarde du graphique en PNG
    plt.savefig(f'capacite_max_{region}.png', dpi=300)
  
    plt.show()

In [ ]:
# ====================AFFICHAGE  DES CAPACITES MAXIMALES RTE ET CALCULEES  PAR FILIERE ET PAR REGION 

In [ ]:
# Liste des chaînes de caractères pour filtrer les colonnes pertinentes
colonnes_cmax = [col for col in df_eco2_tch_rte.columns if 'cmax' in col]
colonnes_cap_prod_max = [col for col in df_eco2_tch_rte.columns if 'cap_prod_max' in col]
colonnes_p_disp = [col for col in df_eco2_tch_rte.columns if 'p_disp' in col]
colonnes_ind_prod = [col for col in df_eco2_tch_rte.columns if 'ind_prod' in col]

# Concaténationn de toutes les colonnes pertinentes
colonnes_pertinentes = colonnes_cmax + colonnes_cap_prod_max + colonnes_p_disp + colonnes_ind_prod

# Filtrage du DataFrame pour inclure uniquement les colonnes pertinentes
df_filtre = df_eco2_tch_rte[['region', 'jour_numero', 'annee'] + colonnes_pertinentes]

# Groupement par région, année et numéro de jour, calcul de la moyenne
df_means = df_filtre.groupby(['region', 'annee', 'jour_numero']).mean().reset_index()

# Création d'une figure et des axes pour les sous-graphiques
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(18, 22))
fig.subplots_adjust(hspace=0.4)

# Mappage des noms de colonnes à des titres compréhensibles
colonnes_filiere = {
    'cmax_thermique_rte': 'Thermique - Cmax',
    'cap_prod_max_thermique': 'Thermique - Cap Prod Max',
    'ind_prod_thermique': 'Thermique - Ind Prod',
    'thermique_p_disp': 'Thermique - P Disp',
    'cmax_nucleaire_rte': 'Nucléaire - Cmax',
    'cap_prod_max_nucleaire': 'Nucléaire - Cap Prod Max',
    'ind_prod_nucleaire': 'Nucléaire - Ind Prod',
    'nucleaire_p_disp': 'Nucléaire - P Disp',
    'cmax_eolien_rte': 'Éolien - Cmax',
    'eolien_p_disp': 'Éolien - P Disp',
    'cmax_solaire_rte': 'Solaire - Cmax',
    'solaire_p_disp': 'Solaire - P Disp',
    'cmax_hydraulique_rte': 'Hydraulique - Cmax',
    'cap_prod_max_hydraulique': 'Hydraulique - Cap Prod Max',
    'ind_prod_hydraulique': 'Hydraulique - Ind Prod',
    'hydraulique_p_disp': 'Hydraulique - P Disp',
    'cmax_bioenergies_rte': 'Bioénergies - Cmax',
    'cap_prod_max_bioenergies': 'Bioénergies - Cap Prod Max',
    'ind_prod_bioenergies': 'Bioénergies - Ind Prod',
    'bioenergies_p_disp': 'Bioénergies - P Disp'
}

# Itération sur  chaque région et chaque filière pour créer les sous-graphiques
for (region, filiere), ax in zip(df_means.groupby(['region', 'annee']), axes.flatten()):
    for col in colonnes_pertinentes:
        # Vérification que la colonne actuelle appartient à la filière actuelle
        if col in colonnes_filiere.keys():
            # Filtrage des données pour la région, l'année et la colonne actuelles
            data = filiere[filiere['region'] == region[0]]
            data = data[data['annee'] == region[1]]

            # Moyenne des valeurs pour chaque jour de numéro
            moyennes_jour = data.groupby('jour_numero')[col].mean()

            # Tracé des courbes avec les moyennes journalières
            ax.plot(moyennes_jour.index, moyennes_jour.values, label=colonnes_filiere[col])

    # Réglages des axes et du titre pour chaque sous-graphique
    ax.set_title(f"{region[0]} - {region[1]}")
    ax.set_xlabel('Numéro de jour')
    ax.set_ylabel('Valeurs moyennes')
    ax.legend()

# Réglages globaux pour l'ensemble de la figure
plt.suptitle('Évolution des valeurs moyennes par filière et par région (2013-2022)', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.97])

# Sauvegarde de chaque sous-graphique individuellement au format PNG avec DPI=300
for i, ax in enumerate(axes.flatten()):
    region_name = df_means['region'].unique()[i][0]
    plt.figure(i)
    plt.savefig(f"Cap_max_{region_name}.png", dpi=300)
  
# Affichage des graphiques
plt.show()

In [ ]:
# VISUALISATION COMPARATIVE AVEC HISTOGRAMMES DES CAPACITES MAXIMALES,PAR REGION - PERIODE 2013-2022

# Définition des filières et les régions
filieres = ['thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'bioenergies']
regions = df_eco2_tch_rte['region'].unique()

# Définition des colonnes à comparer pour chaque filière
colonnes_par_filiere = {
    'thermique': ['cmax_thermique_rte', 'cap_prod_max_thermique', 'ind_prod_thermique', 'thermique_p_disp'],
    'nucleaire': ['cmax_nucleaire_rte', 'cap_prod_max_nucleaire', 'ind_prod_nucleaire', 'nucleaire_p_disp'],
    'eolien': ['cmax_eolien_rte', 'eolien_p_disp'],
    'solaire': ['cmax_solaire_rte', 'solaire_p_disp'],
    'hydraulique': ['cmax_hydraulique_rte', 'cap_prod_max_hydraulique', 'ind_prod_hydraulique', 'hydraulique_p_disp'],
    'bioenergies': ['cmax_bioenergies_rte', 'cap_prod_max_bioenergies', 'ind_prod_bioenergies', 'bioenergies_p_disp']
}

# Filtrage du dataframe pour les années 2013 à 2022
df_filtered = df_eco2_tch_rte[df_eco2_tch_rte['annee'].between(2013, 2022)]

# Création des graphiques à barres groupées pour chaque région et chaque filière
for region in regions:
    fig, axes = plt.subplots(nrows=len(filieres), ncols=1, figsize=(12, 15), sharex=True)
    fig.suptitle(f'Comparaison des valeurs des capacités de production, période 2013-2022, pour la région {region}', fontsize=16)
    
    for i, filiere in enumerate(filieres):
        # Sélection des colonnes à comparer pour la filière spécifique
        colonnes_a_comparer = colonnes_par_filiere[filiere]
        
        # Sélection des données pour la région et la filière spécifiques
        df_region_filiere = df_filtered[(df_filtered['region'] == region)]
        
        # Préparation des données pour le graphique à barres groupées
        annees = df_region_filiere['annee'].unique()
        donnees = {colonne: df_region_filiere.groupby('annee')[colonne].mean() for colonne in colonnes_a_comparer}
        
        # Création du graphique à barres groupées pour cette filière
        largeur_barre = 0.2
        indices = np.arange(len(annees))
        
        for j, (nom_colonne, serie) in enumerate(donnees.items()):
            axes[i].bar(indices + j * largeur_barre, serie, width=largeur_barre, label=nom_colonne)
        
        # Configuration des étiquettes et légendes pour chaque subplot
        axes[i].set_ylabel('Valeurs')
        axes[i].set_title(f'Filière {filiere}')
        axes[i].legend(loc='upper left', bbox_to_anchor=(1, 1))  # Déplacer la légende en dehors du sous-graphe
        
    # Configuration des étiquettes de l'axe des x pour le dernier subplot
    axes[-1].set_xticks(indices + (len(colonnes_a_comparer) - 1) * largeur_barre / 2)
    axes[-1].set_xticklabels(annees)
    axes[-1].set_xlabel('Année')
    
    # Ajustement de la disposition du graphique pour minimiser les chevauchements
    plt.tight_layout(rect=[0, 0, 0.85, 0.96])  # Ajuster l'espace pour les légendes en dehors des sous-graphes
    
    # Exportation du graphique en fichier image
    nom_fichier = f'hist_comparaison_rte_cal_{region}.png'
    plt.savefig(nom_fichier, bbox_inches='tight')
    
    # Affichage du graphique pour cette région
    plt.show()


print("Export des graphiques terminé.")